In [4]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import networkx as nx
!pip install -q stellargraph
!pip install gensim


from mpl_toolkits.mplot3d import Axes3D
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 435 kB 7.9 MB/s 
Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

#loading data
isolated_nodes = pd.read_csv("isolated_nodes.csv")
node_classification = pd.read_csv("node_classification.csv")      #there are 4 unique page types
training_graph = pd.read_csv("training_graph.csv")
with open('node_features_text.json') as f:
   node_features_text = json.load(f)
#Getting IDs for nodes that are of type 4

#change to np array for graph
tg_np = training_graph.to_numpy()

G = nx.Graph()
percent = 0

#generating the graph, keeping track of percent
for i in range(np.shape(tg_np)[0]):
  percent = percent + 1
  print(str(percent * 100 / np.shape(tg_np)[0])[0:4] + "%")
  G.add_edge(tg_np[i][0], tg_np[i][1])

print(nx.info(G))

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training


In [5]:
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
import multiprocessing

In [6]:
#sourced from https://stellargraph.readthedocs.io/en/v1.0.0rc1/demos/link-prediction/random-walks/cora-lp-demo.html#Node2Vec and Bill.com Workshop

#variables for walks and node2vec
p = 1.0
q = 1.0 
walk_length = 100
num_walks = 10
dimensions = 100
window_size = 10
sg = 1
workers = multiprocessing.cpu_count()

def gen_walks(G):
  """
  Returns randomly generate walks from a networkx graph

  Inputs - G: A networkx graph
  Outputs - walks: List of lists, where each inner list is a walk path
  """
  #change to stellargraph and run BiasedRandomWalk with equal percentages
  SG = StellarGraph.from_networkx(G)
  rw = BiasedRandomWalk(SG)

  #generate walks
  walks = rw.run(SG.nodes(), n=num_walks, length=walk_length, p=p, q=q)

  print("Number of random walks: {}".format(len(walks)))
  return walks

def node2vec_model(walks):
  """
  Generates node2vec model given a walk path

  Inputs - walks: list of lists, where each inner list is a walk path
  Outputs - model: Word2Vec model
  """
  model = Word2Vec(
      walks,
      size=dimensions,
      window=window_size,
      min_count=0,
      sg=1,
      workers=workers,
      iter=1)
  
  return model
  

In [ ]:
#generate our walk paths
walks = gen_walks(G)

Number of random walks: 208160


In [ ]:
#convert to dataframe and save so we don't have to rerun
walks_df = pd.DataFrame(walks)
walks_df.to_csv("node2vec_walks.csv")

In [ ]:
#check if it looks good
print(walks_df.head())

      0     1      2      3      4   ...     95     96     97     98    99
0  17807  2105  11209   8204  14502  ...  14652  17480  15049  20500   695
1  17807  3897    396   5913    396  ...   1243   9128  12461  19515  4909
2  17807  2632  18427  19189  16054  ...  13613   6666  10882   8745  2813
3  17807  2632  20845   9009  20845  ...   7357  19481   3361   7357  9678
4  17807  2632  20690   3652  11581  ...   7357   9554  16226   3878  8351

[5 rows x 100 columns]


In [ ]:
#Go through walks and change each inner node, represented by an int, to a string
new_walks = []
i = 0
total = len(walks)
for inwalks in walks:
  print(str(i * 100 / len(walks))[0:4])
  new_walks.append([str(n) for n in inwalks])
  i += 1

In [ ]:
#generates model
model = node2vec_model(new_walks)

In [ ]:
#check if dimensions look right
print(model)

Word2Vec(vocab=20816, size=100, alpha=0.025)


In [ ]:
#generate a final data matrix; if the node is not isolated, append the given data.
#if it is isolated, represent the node2vec representation as 100 zeros
isolated = isolated_nodes.to_numpy().flatten()
finlist = np.zeros((22470, 100))
for i in range(22470):
  if i not in isolated:
    finlist[i,:] = model[str(i)]
  else:
    finlist[i, :] = np.zeros(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
#save to df
finlist_df = pd.DataFrame(finlist)
print(finlist_df.head)

<bound method NDFrame.head of              0         1         2   ...        97        98        99
0      0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
1      0.615414  0.090006 -0.213775  ...  0.069782  0.457586 -0.023602
2      0.245422  0.073416  0.933528  ... -0.220473  0.366260 -0.383944
3      0.549525 -0.212722 -0.016863  ...  0.477046  0.341830 -0.405520
4      0.107111  0.744169  0.132202  ...  0.290825  0.096730  0.047691
...         ...       ...       ...  ...       ...       ...       ...
22465  0.360505  0.094510  0.712458  ...  0.317086  0.101235  0.545431
22466 -0.009267  0.495264  0.712387  ...  0.248221 -0.061972 -0.217918
22467  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
22468  0.395354  0.043508  0.015739  ...  0.413606 -0.108394  0.058372
22469  0.304754 -0.452971  0.487920  ... -0.253164  0.357887 -0.126110

[22470 rows x 100 columns]>


In [ ]:
#save dataframe
finlist_df.to_csv("node2vec_features.csv")

In [ ]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
 
 #generating walks for our testing data
testg = pd.read_csv("test_edges.csv")
testg_np = testg.to_numpy()

#generation of the graph
G = nx.Graph()
percent = 0
for i in range(np.shape(testg_np)[0]):
  percent = percent + 1
  print(str(percent * 100 / np.shape(testg_np)[0])[0:4] + "%")
  G.add_edge(testg_np[i][0], testg_np[i][1])

print(nx.info(G))

In [ ]:
#generating walks for testing data
walks = gen_walks(G)

Number of random walks: 224700


In [ ]:
#saving to dataframe
walks_df = pd.DataFrame(walks)
walks_df.to_csv("testing_walks.csv")

In [ ]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
walks = np.array(pd.read_csv("testing_walks.csv"))

#same process as before, change each int representing a node to a string for node2vec
new_walks = []
i = 0
total = len(walks)
for inwalks in walks:
  print(str(i * 100 / len(walks))[0:4])
  new_walks.append([str(n) for n in inwalks])
  i += 1

In [9]:
#generate model with testing walks
model = node2vec_model(new_walks)

In [11]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

#loading data
isolated_nodes = pd.read_csv("isolated_nodes.csv")
node_classification = pd.read_csv("node_classification.csv")      #there are 4 unique page types
training_graph = pd.read_csv("training_graph.csv")
with open('node_features_text.json') as f:
   node_features_text = json.load(f)
#Getting IDs for nodes that are of type 4

#generate our data matrix; if the node is isolated represent the node2vec vector as 100 zeros
isolated = isolated_nodes.to_numpy().flatten()
finlist = np.zeros((22470, 100))
for i in range(22470):
  if i not in isolated:
    finlist[i,:] = model[str(i)]
  else:
    finlist[i, :] = np.zeros(100)

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [ ]:
#create df
finlist_df = pd.DataFrame(finlist)
print(finlist_df.head)

In [14]:
#save df
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
finlist_df.to_csv("testing_node2vec.csv")

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Test


In [ ]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training
training_graph = pd.read_csv("training_graph.csv")
tg_np = training_graph.to_numpy()

#generate our graph from our training data again
G = nx.Graph()
percent = 0
for i in range(np.shape(tg_np)[0]):
  percent = percent + 1
  print(str(percent * 100 / np.shape(tg_np)[0])[0:4] + "%")
  G.add_edge(tg_np[i][0], tg_np[i][1])

print(nx.info(G))

In [101]:
import random
import math

def remove_edges(graph, p):
  """
  Takes a networkx graph and randomly removes p% of the edges

  INPUTS - graph: a networkx graph
           p: proportion of edges to remove
  OUTPUTS - graph: a sparsified version of the original graph
  """
  #generate a random sample of the edges
  edges = list(graph.edges)
  edges_remove = random.sample(edges, math.floor(len(edges)*p))

  #for each edge, if the nodes are both not degree 1, as in they do not have only one 
  #connection with the other to avoid unconnected nodes, remove the edge
  for edge in edges_remove:
    if ((graph.degree[edge[0]] != 1) and (graph.degree[edge[1]] != 1)):
      graph.remove_edge(edge[0], edge[1])
  
  return graph

In [103]:
#get total edges for reference
total_edges = len(list(G.edges))
print(f"Total Edges: {total_edges}")

#make a copy of the graph
copy_g = G.copy()
sparse_g = remove_edges(graph=copy_g, p=.5)

#check how much was removed
ratio = str(100 - len(sparse_g.edges) * 100/ total_edges)[0:4]
print(f"Removed {ratio}% of edges")

Total Edges: 132038
Removed 48.2% of edges


In [104]:
print(nx.info(sparse_g))
#check the size if the resulting graphs

Graph with 20816 nodes and 68341 edges


In [105]:
training_graph = list((G.subgraph(c) for c in nx.connected_components(G)))[0]
print(nx.info(training_graph))
#check size of original graph G

Graph with 20589 nodes and 131840 edges


In [106]:
training_graph = list((copy_g.subgraph(c) for c in nx.connected_components(copy_g)))[0]
print(nx.info(training_graph))
#check size of copy_g of G

Graph with 19099 nodes and 67035 edges


In [107]:
training_graph = list((sparse_g.subgraph(c) for c in nx.connected_components(sparse_g)))[0]
print(nx.info(training_graph))
#check size of sparse_g, the resulting graph

#we are losing nodes somewhere, even with degree checking. Not enough time to debug

Graph with 19099 nodes and 67035 edges


In [108]:
#generate our walks, change to 5 since our graph is half of the edges, so should still 
#capture the same amount of information with more sparse nodes
num_walks = 5
walks = gen_walks(sparse_g)

Number of random walks: 104080


In [109]:
#save to csv
walks_df = pd.DataFrame(walks)
walks_df.to_csv("sparse_walks.csv")

In [ ]:
#same as before, change ints to strings for node2vec
new_walks = []
i = 0
total = len(walks)
for inwalks in walks:
  print(str(i * 100 / len(walks))[0:4])
  new_walks.append([str(n) for n in inwalks])
  i += 1

model = node2vec_model(new_walks)

In [111]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

#generate our final matrix where isolated nodes are represented as 100 zeros rather
#than node2vec

isolated = isolated_nodes.to_numpy().flatten()
finlist = np.zeros((22470, 100))
for i in range(22470):
  if i not in isolated:
    finlist[i,:] = model[str(i)]
  else:
    finlist[i, :] = np.zeros(100)

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [112]:
#get our final dataframe
finlist_df = pd.DataFrame(finlist)
print(finlist_df.head)

<bound method NDFrame.head of              0         1         2   ...        97        98        99
0      0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
1     -0.221936 -0.618475  0.157521  ... -0.430317  0.341653  0.015302
2      0.218412 -0.916328 -0.851940  ... -0.311026  0.279659 -0.018877
3     -0.024505  0.328660  0.539155  ...  0.195045 -0.696718 -0.384063
4     -0.012618 -0.605725 -0.121598  ...  0.065442  0.043176 -0.044069
...         ...       ...       ...  ...       ...       ...       ...
22465 -0.171671 -0.376132 -0.101205  ... -0.502479  0.147777  0.310219
22466  0.328381  0.097097 -0.310297  ... -0.186292  0.093544 -0.339893
22467  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000
22468  0.042153 -0.027671 -0.397323  ...  0.074028  0.016360 -0.128527
22469  0.106251 -0.725376 -0.350690  ... -0.464668 -0.282102  0.342022

[22470 rows x 100 columns]>


In [113]:
#save to csv
finlist_df.to_csv("sparse_node2vec.csv")